You work for a telecom company that wants to analyze how customers use data, calls, and SMS based on their subscribed plan.

In [1]:
import pandas as pd

plans = pd.DataFrame([
    {"plan_id": "P101", "plan_name": "Super Saver", "plan_type": "Prepaid", "data_limit_gb": 15, "call_limit": 300, "sms_limit": 100, "monthly_cost": 299},
    {"plan_id": "P102", "plan_name": "Unlimited Pro", "plan_type": "Postpaid", "data_limit_gb": 50, "call_limit": 2000, "sms_limit": 500, "monthly_cost": 699},
    {"plan_id": "P103", "plan_name": "Data Boost", "plan_type": "Prepaid", "data_limit_gb": 10, "call_limit": 200, "sms_limit": 50, "monthly_cost": 249},
    {"plan_id": "P104", "plan_name": "Family Connect", "plan_type": "Postpaid", "data_limit_gb": 30, "call_limit": 1500, "sms_limit": 400, "monthly_cost": 599},
    {"plan_id": "P105", "plan_name": "Youth Max", "plan_type": "Prepaid", "data_limit_gb": 25, "call_limit": 500, "sms_limit": 200, "monthly_cost": 399},
    {"plan_id": "P106", "plan_name": "Business Elite", "plan_type": "Postpaid", "data_limit_gb": 100, "call_limit": 5000, "sms_limit": 1000, "monthly_cost": 999},
    {"plan_id": "P107", "plan_name": "Weekend Binge", "plan_type": "Prepaid", "data_limit_gb": 20, "call_limit": 300, "sms_limit": 150, "monthly_cost": 349},
    {"plan_id": "P108", "plan_name": "Premium 5G", "plan_type": "Postpaid", "data_limit_gb": 150, "call_limit": 5000, "sms_limit": 1200, "monthly_cost": 1199},
    {"plan_id": "P109", "plan_name": "Student Connect", "plan_type": "Prepaid", "data_limit_gb": 12, "call_limit": 250, "sms_limit": 80, "monthly_cost": 279},
    {"plan_id": "P110", "plan_name": "Corporate Flex", "plan_type": "Postpaid", "data_limit_gb": 200, "call_limit": 8000, "sms_limit": 2000, "monthly_cost": 1499},
    {"plan_id": "P111", "plan_name": "Rural Value", "plan_type": "Prepaid", "data_limit_gb": 8, "call_limit": 150, "sms_limit": 50, "monthly_cost": 199},
    {"plan_id": "P112", "plan_name": "5G Ultra Unlimited", "plan_type": "Postpaid", "data_limit_gb": 999, "call_limit": 10000, "sms_limit": 5000, "monthly_cost": 1999}
])

plans.to_csv("plans.csv", index=False)
print("✅ plans.csv saved with", len(plans), "records.")

✅ plans.csv saved with 12 records.


In [2]:
pip install sqlalchemy

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install pymysql

Note: you may need to restart the kernel to use updated packages.


In [5]:
import pandas as pd
from sqlalchemy import create_engine

# ✅ MySQL connection setup
user = 'root'
password = 'RAJATVEER'   # Your MySQL password
host = '127.0.0.1'        # Localhost
port = '3306'             # Default MySQL port
database = 'telecomdb'    # ✅ Correct database name

# ✅ Create the SQLAlchemy engine
engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}:{port}/{database}")

# ✅ Test the connection

with engine.connect() as connection:
        print("✅ Connected to MySQL database 'telecomdb' successfully!")



✅ Connected to MySQL database 'telecomdb' successfully!


In [10]:

usage = pd.read_sql("SELECT * FROM usage_data", engine)

plans = pd.read_csv("plans.csv")

print("Rows extracted -> Usage:", len(usage), "Plans:", len(plans))
display(usage.head(), plans.head())


Rows extracted -> Usage: 20 Plans: 12


,usage_id,customer_id,plan_id,month,data_used_gb,calls_made,sms_sent,region
0,1,1001,P101,September,12.5,210,45,Delhi
1,2,1002,P102,September,18.3,340,30,Mumbai
2,3,1003,P103,September,7.8,120,55,Chennai
3,4,1004,P101,September,25.2,410,20,Delhi
4,5,1005,P104,September,9.6,200,15,Kolkata


,plan_id,plan_name,plan_type,data_limit_gb,call_limit,sms_limit,monthly_cost
0,P101,Super Saver,Prepaid,15,300,100,299
1,P102,Unlimited Pro,Postpaid,50,2000,500,699
2,P103,Data Boost,Prepaid,10,200,50,249
3,P104,Family Connect,Postpaid,30,1500,400,599
4,P105,Youth Max,Prepaid,25,500,200,399


In [7]:
# --- Standardize text ---
usage['region'] = usage['region'].str.title().str.strip()
plans['plan_type'] = plans['plan_type'].str.title().str.strip()

# --- Convert numeric columns ---
usage['data_used_gb'] = pd.to_numeric(usage['data_used_gb'], errors='coerce')
plans['monthly_cost'] = pd.to_numeric(plans['monthly_cost'], errors='coerce')

# --- Handle missing or invalid data ---
usage['data_used_gb'] = usage['data_used_gb'].fillna(0)
usage['calls_made'] = usage['calls_made'].fillna(0)
usage['sms_sent'] = usage['sms_sent'].fillna(0)

# --- Merge datasets ---
merged = usage.merge(plans, on='plan_id', how='left')

# --- Create derived columns ---
merged['data_exceed_flag'] = merged['data_used_gb'] > merged['data_limit_gb']
merged['call_exceed_flag'] = merged['calls_made'] > merged['call_limit']
merged['sms_exceed_flag'] = merged['sms_sent'] > merged['sms_limit']

print("✅ Data transformed successfully!")
display(merged.head())


✅ Data transformed successfully!


,usage_id,customer_id,plan_id,month,data_used_gb,calls_made,sms_sent,region,plan_name,plan_type,data_limit_gb,call_limit,sms_limit,monthly_cost,data_exceed_flag,call_exceed_flag,sms_exceed_flag
0,1,1001,P101,September,12.5,210,45,Delhi,Super Saver,Prepaid,15,300,100,299,False,False,False
1,2,1002,P102,September,18.3,340,30,Mumbai,Unlimited Pro,Postpaid,50,2000,500,699,False,False,False
2,3,1003,P103,September,7.8,120,55,Chennai,Data Boost,Prepaid,10,200,50,249,False,False,True
3,4,1004,P101,September,25.2,410,20,Delhi,Super Saver,Prepaid,15,300,100,299,True,True,False
4,5,1005,P104,September,9.6,200,15,Kolkata,Family Connect,Postpaid,30,1500,400,599,False,False,False


In [8]:
merged.to_csv("usage_plan_output.csv", index=False)
print(f"✅ ETL pipeline complete! Created {len(merged)} records and saved usage_plan_output.csv.")


✅ ETL pipeline complete! Created 20 records and saved usage_plan_output.csv.


QUICK DATA ANALYSIS

In [9]:
print("\nTop Data Users (Above Limit):")
print(merged[merged['data_exceed_flag']].groupby(['region','plan_type']).customer_id.count().reset_index(name='count'))

print("\nAverage Data Usage by Plan Type:")
print(merged.groupby('plan_type').data_used_gb.mean().reset_index(name='avg_data_gb'))

print("\nUsage Summary by Region:")
print(merged.groupby('region')[['data_used_gb','calls_made','sms_sent']].mean().reset_index())



Top Data Users (Above Limit):
  region plan_type  count
0  Delhi   Prepaid      2
1   Pune   Prepaid      1

Average Data Usage by Plan Type:
  plan_type  avg_data_gb
0  Postpaid    22.755556
1   Prepaid    12.436364

Usage Summary by Region:
      region  data_used_gb  calls_made   sms_sent
0    Chennai      6.766667  113.333333  33.333333
1      Delhi     16.400000  270.000000  39.000000
2  Hyderabad     11.066667  190.000000  41.666667
3    Kolkata     14.000000  260.000000  18.333333
4     Mumbai     27.500000  486.666667  41.666667
5       Pune     27.200000  460.000000  53.333333
